In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from DataPreprocess import create_dataloader
from InitializingModule import InitModel
from TrainingAlgorithm import train

In [2]:
train_loader, val_loader = create_dataloader(batch_size=128)

In [3]:
device = torch.device('cuda')
a = InitModel(12, 1, 1, device, batch_norm=True)
test_model = a.init_module_1_ReLU_LR(train_loader)
optimizer = optim.Adam(test_model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

In [23]:
model = train(train_loader=train_loader, val_loader=val_loader, model=test_model, epochs=100, optimizer=optimizer, criterion=criterion, device=device)

[ 1/100 ] | train_loss = 0.61232, train_acc = 0.66927, val_loss = 0.56571, val_acc = 0.82292
[ 2/100 ] | train_loss = 0.60456, train_acc = 0.66927, val_loss = 0.59488, val_acc = 0.82292
[ 3/100 ] | train_loss = 0.59577, train_acc = 0.67448, val_loss = 0.58164, val_acc = 0.82292
[ 4/100 ] | train_loss = 0.58541, train_acc = 0.69010, val_loss = 0.58267, val_acc = 0.84375
[ 5/100 ] | train_loss = 0.58005, train_acc = 0.68229, val_loss = 0.54317, val_acc = 0.85417
[ 6/100 ] | train_loss = 0.56904, train_acc = 0.69271, val_loss = 0.54852, val_acc = 0.83333
[ 7/100 ] | train_loss = 0.56369, train_acc = 0.67969, val_loss = 0.56546, val_acc = 0.82292
[ 8/100 ] | train_loss = 0.55744, train_acc = 0.71875, val_loss = 0.58816, val_acc = 0.81250
[ 9/100 ] | train_loss = 0.54866, train_acc = 0.72917, val_loss = 0.55414, val_acc = 0.80208
[ 10/100 ] | train_loss = 0.54615, train_acc = 0.71875, val_loss = 0.56103, val_acc = 0.80208
[ 11/100 ] | train_loss = 0.53423, train_acc = 0.74219, val_loss = 0.

In [13]:
test_2 = []
for index, item in enumerate(test):
    if item < 3:
        test_2.append([index, item])

In [14]:
test_2

[[0, 0], [1, 1], [2, 2], [5, 0], [6, 1], [7, 2]]

In [40]:
model.eval()
valid_loss = []

for i, batch in enumerate(train_loader):
    x, y = batch
    print(x.shape)
    print(y.shape)
    with torch.no_grad():
        logits = model(x.to(device))
        loss = criterion(logits, y.to(device).unsqueeze(1))
        valid_loss.append(loss.item())

torch.Size([128, 12])
torch.Size([128])
torch.Size([128, 12])
torch.Size([128])
torch.Size([128, 12])
torch.Size([128])


In [26]:
valid_loss

[0.3376418948173523, 0.3637465238571167, 0.36652877926826477]

In [87]:
model.eval()
valid_loss = []


for i, data in enumerate(train_loader.dataset):
    x, y = data[0].view(-1, 12), data[1].view(-1)
    with torch.no_grad():
        logits = model(x.to(device))
        loss = criterion(logits, y.to(device).unsqueeze(1))
        valid_loss.append((i, loss.item()))

In [94]:
picked_loss = []
for item in valid_loss:
    if item[1] < 0.0273:
        picked_loss.append(item)

In [95]:
picked_loss

[(469, 0.018786944448947906),
 (365, 0.019591515883803368),
 (369, 0.019591515883803368),
 (372, 0.019591515883803368),
 (373, 0.019591515883803368),
 (479, 0.02270965650677681),
 (474, 0.023401854559779167),
 (366, 0.025850633159279823),
 (470, 0.026753077283501625),
 (477, 0.027251092717051506)]

In [100]:
index = [i for i, item in picked_loss]

In [88]:
valid_loss.sort(key=lambda x: x[1])

In [91]:
valid_loss[:10]

[(469, 0.018786944448947906),
 (365, 0.019591515883803368),
 (369, 0.019591515883803368),
 (372, 0.019591515883803368),
 (373, 0.019591515883803368),
 (479, 0.02270965650677681),
 (474, 0.023401854559779167),
 (366, 0.025850633159279823),
 (470, 0.026753077283501625),
 (477, 0.027251092717051506)]

In [89]:
len(valid_loss[:160])

160

In [71]:
len(train_loader.dataset)

480

In [110]:
from torch.utils.data import Subset, DataLoader

In [103]:
new_train_dataset = Subset(train_loader.dataset, index)

In [111]:
new_dataloader = DataLoader(new_train_dataset, train_loader.batch_size)

In [113]:
next(iter(new_dataloader))

[tensor([[-0.0920, -0.0428, -0.8002, -0.7843,  2.5858,  2.3229, -1.0560, -0.6572,
           2.3386,  1.0564,  2.1492,  2.0137],
         [-0.3663, -0.2966,  2.3515, -0.5910,  0.1528,  0.6837, -0.7335, -0.5271,
           0.6465,  1.8232, -2.6880, -2.0541],
         [-1.0925, -0.7333, -3.5575,  1.1864,  0.2222,  0.7407, -0.9812, -0.0716,
           0.1605,  3.5938, -5.8611, -4.5000],
         [-0.4874, -0.1342, -0.1102, -3.7465,  0.3704,  0.8449, -1.1786,  1.4901,
          -0.0626,  2.2530, -1.8190, -4.5119],
         [-0.6326, -0.2357,  0.2342, -1.2005,  0.3343,  0.8533, -1.1618,  1.7504,
          -0.2044,  2.5882, -3.0543, -5.0122],
         [ 0.5696,  0.4853, -0.2694,  0.4319,  2.9502,  2.5087, -0.9019, -1.3079,
           2.8226,  0.9471,  1.2030,  2.1932],
         [ 0.4324,  0.5462, -0.6352, -0.4485,  2.8277,  2.4056, -0.9381, -0.9175,
           2.5051,  0.8889,  0.9921,  1.7321],
         [-0.4793, -0.3373,  1.3179, -1.0128,  0.1658,  0.6876, -0.7311, -0.2668,
           0.62

In [108]:
new_train_dataset[0]

(tensor([-0.0920, -0.0428, -0.8002, -0.7843,  2.5858,  2.3229, -1.0560, -0.6572,
          2.3386,  1.0564,  2.1492,  2.0137], device='cuda:0'),
 tensor(1., device='cuda:0'))

In [77]:
index = [i for i, item in valid_loss]


In [109]:
train_loader.batch_size

128

In [86]:
len(index)

480

In [85]:
train_loader.dataset[index]

(tensor([[-0.0920, -0.0428, -0.8002,  ...,  1.0564,  2.1492,  2.0137],
         [-0.3663, -0.2966,  2.3515,  ...,  1.8232, -2.6880, -2.0541],
         [-1.0925, -0.7333, -3.5575,  ...,  3.5938, -5.8611, -4.5000],
         ...,
         [-0.9473, -0.2052, -0.8941,  ...,  1.5301, -0.9284,  0.6381],
         [-0.8827, -0.5505,  0.4689,  ..., -0.0167, -1.3415, -1.6768],
         [-1.0844, -0.4693,  0.0914,  ...,  1.4068, -0.1677,  1.2303]],
        device='cuda:0'),
 tensor([1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
         1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
         0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1.